### Scoring Algorithm

In [2]:
import pandas as pd

try:
    #Import applicant data
    df = pd.read_csv('applicants.csv')

    #Number of applicants to select (ITERATION 2: MAKE DYNAMIC)
    N = 5

    #Clean column names
    df.columns = df.columns.str.strip()

    #Copy for scoring
    df_score = df.copy()

    #Transform binary columns for scoring
    df_score['Past Volunteer (score)'] = df_score['Past Volunteer'].map({'Yes': 2, 'No': 0})
    df_score['Past Camper (score)'] = df_score['Past Camper'].map({'Yes': 1, 'No': 0})

    #Add count columns (keep raw columns intact)
    df_score['Course Count'] = df_score['Courses'].apply(lambda x: len(str(x).split(';')))
    df_score['Technical Skill Count'] = df_score['Technical Skills'].apply(lambda x: len(str(x).split(';')))

    #Map experience columns
    experience_map = {'0 hrs': 0, '1-9 hrs': 1, '10-20 hrs': 2, '20+ hrs': 3}
    df_score['Youth Experience (score)'] = df_score['Youth Experience'].map(experience_map)
    df_score['Leadership Experience (score)'] = df_score['Leadership Experience'].map(experience_map)

    #Calculate score based on counts and scoring columns
    df_score['Score'] = (
        df_score['Past Volunteer (score)'] +
        df_score['Past Camper (score)'] +
        df_score['Course Count'] +
        df_score['Technical Skill Count'] +
        df_score['Youth Experience (score)'] +
        df_score['Leadership Experience (score)']
    )

    #Sort by score and assign rank
    df_score = df_score.sort_values(by='Score', ascending=False).reset_index(drop=True)
    df_score.insert(0, 'Rank', df_score.index + 1)

    #Top N scorers
    df_score_top = df_score.head(N)

    #Save outputs
    df_score_top.to_csv("top_" + str(N) + "_applicants.csv", index=False)
except:
    print('Something went wrong. Please check the input file and try again.')


### Scheduling Optimization

In [3]:
#Import relevant data (Qualtrics formatted export)
availability_df = pd.read_csv('Capstone Sample Weekly Scheduling Data.csv', skiprows=[0,1])

#Clean column names and format
new_header = ['Name', 'Email', 'Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6', 'Week 7']
availability_df.columns = new_header
availability_df.insert(0, 'Volunteer ID', availability_df.index + 1)
availability_map = {'Yes': 1, 'No': 0}

for col in availability_df.columns[3:]:
    availability_df[col] = availability_df[col].map(availability_map)

#Sample weekly volunteer requirements N_j (ITERATION 2: MAKE DYNAMIC)
weekly_volunteers = [3,2,4,5,4,3,2]

#availability_df

In [9]:
#Import Gurobi for optimization
import gurobipy as gp
from gurobipy import GRB

#Create model
m = gp.Model("Scheduling Optimization")

#Set volunteers and week numerated list
volunteers = availability_df['Volunteer ID'].tolist()
weeks = list(range(1,8))

#Set decision variable
week_match = m.addVars(volunteers, weeks, vtype=GRB.BINARY, name="x")

#Count available weeks per volunteer
availability_counts = {
    i: sum(availability_df.loc[
        availability_df['Volunteer ID'] == i,
        availability_df.columns[3:]   # week columns
    ].values[0])
    for i in volunteers
}

#Create priority weights
max_availability = max(availability_counts.values())

weights = {
    i: (max_availability - availability_counts[i] + 1)
    for i in volunteers
}

#Set constraints
for i in volunteers:
    m.addConstr(gp.quicksum(week_match[i,j] for j in weeks) >= 1, name=f'Volunteer_{i}_Min_Placement')

for j in weeks:
    m.addConstr(gp.quicksum(week_match[i,j] for i in volunteers) == weekly_volunteers[j-1], name=f'Week_{j}_Min_Volunteers')

for i in volunteers:
    for j in weeks:
        m.addConstr(week_match[i,j] <= availability_df.iloc[i-1,j+2], name=f'Availability_{i}_{j}')

#Set objective function
m.setObjective(
    gp.quicksum(
        weights[i] * week_match[i, j]
        for i in volunteers
        for j in weeks
    ),
    GRB.MAXIMIZE
)

#Optimize model
m.optimize()

'''
#comment out, for testing purposes
if m.Status == GRB.OPTIMAL:
    print(f"\nOptimal Objective Value: {m.ObjVal}\n")
    print("Assigned Volunteers per Week:")
    for i in volunteers:
        for j in weeks:
            if week_match[i, j].X > 0.5:
                print(f" - Volunteer {i} assigned to Week {j}")
                print(week_match[i, j].X)
else:
    print("No optimal solution found.")
'''

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 87 rows, 70 columns and 210 nonzeros
Model fingerprint: 0x29ab3531
Variable types: 0 continuous, 70 integer (70 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 54.0000000
Presolve removed 83 rows and 63 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 12 nonzeros
Found heuristic solution: objective 60.0000000
Variable types: 0 continuous, 7 integer (7 binary)
Found heuristic solution: objective 63.0000000

Root relaxation: objective 6.400000e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      

'\n#comment out, for testing purposes\nif m.Status == GRB.OPTIMAL:\n    print(f"\nOptimal Objective Value: {m.ObjVal}\n")\n    print("Assigned Volunteers per Week:")\n    for i in volunteers:\n        for j in weeks:\n            if week_match[i, j].X > 0.5:\n                print(f" - Volunteer {i} assigned to Week {j}")\n                print(week_match[i, j].X)\nelse:\n    print("No optimal solution found.")\n'

### Export Results

In [10]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill

#Create workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Schedule"

#Cols for each week
cell_mapping = [('A', 'B'), ('C', 'D'), ('E', 'F'), ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'N')]

#Style
header_font = Font(bold=True)
header_fill = PatternFill(start_color="daa2fc", end_color="daa2fc", fill_type="solid")

#Fill weekly lists with names and emails
week_number = 1
for start_cell, end_cell in cell_mapping:
    ws.merge_cells(start_cell + str(1) + ':' + end_cell + str(1))
    cell = ws[start_cell + str(1)]
    cell.value = f"Week {week_number}"
    cell.font = header_font
    cell.fill = header_fill
    row = 2

    for i in volunteers:
        if week_match[i, week_number].X > 0.5:
            ws[f"{start_cell}{row}"] = availability_df.loc[availability_df['Volunteer ID'] == i, 'Name'].values[0]
            ws[f"{end_cell}{row}"] = availability_df.loc[availability_df['Volunteer ID'] == i, 'Email'].values[0]

            row += 1
    week_number += 1

#Save file
wb.save("Volunteer_Schedule.xlsx")
